In [58]:
import pandas as pd
from sklearn.multioutput import RegressorChain
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [59]:
df_envi_inter = pd.read_csv("../../../DataSet/Envi/envi_inter.csv")
df_envi_inter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    169 non-null    object
 1   Envi    169 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.8+ KB


In [60]:
df_envi_inter

,Date,Envi
0,04/01/2015,44562
1,11/01/2015,38112
2,18/01/2015,50355
3,25/01/2015,51922
4,01/02/2015,67496
...,...,...
164,25/02/2018,42116
165,04/03/2018,44454
166,11/03/2018,28399
167,18/03/2018,49145


In [61]:
def findBestRandomState(r,t, x, y):
    random = 0
    a = 1000000

    model = RegressorChain(GradientBoostingRegressor())
    for _ in range(r,t):
        x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=_)
        model.fit(x_train, y_train)

        y_predict = model.predict(x_test)
        if (buff := mean_absolute_error(y_test, y_predict)) < a:
            a = buff
            random = _
    print(a, random)

Function Create X Sequence to Prdict 1

In [62]:
def createSequenceToOneDataSet(r, data:pd.DataFrame):  # Create Sequence DataSet
    df = data.copy()
    i = 1
    
    while i < r:
        df[f'x{i}'] = df['Envi'].shift(-i)
        i += 1

    if i == r:
        df['y'] = df['Envi'].shift(-i)

    df = df.dropna(axis=0)
    return df

DataSet for Predict y by Use Sequence X

In [63]:
df = createSequenceToOneDataSet(5, df_envi_inter)
df

,Date,Envi,x1,x2,x3,x4,y
0,04/01/2015,44562,38112.0,50355.0,51922.0,67496.0,54229.0
1,11/01/2015,38112,50355.0,51922.0,67496.0,54229.0,41548.0
2,18/01/2015,50355,51922.0,67496.0,54229.0,41548.0,50872.0
3,25/01/2015,51922,67496.0,54229.0,41548.0,50872.0,40578.0
4,01/02/2015,67496,54229.0,41548.0,50872.0,40578.0,46025.0
...,...,...,...,...,...,...,...
159,21/01/2018,49711,66654.0,57481.0,48309.0,39773.0,42116.0
160,28/01/2018,66654,57481.0,48309.0,39773.0,42116.0,44454.0
161,04/02/2018,57481,48309.0,39773.0,42116.0,44454.0,28399.0
162,11/02/2018,48309,39773.0,42116.0,44454.0,28399.0,49145.0


In [64]:
x = df[['Envi','x1','x2','x3','x4']].values # Pasing Df_x to x
y = df['y'].values                          # Pasing Df_y to y

y = y.reshape(-1,1)
# scaler = StandardScaler()
# scaler.fit(x)
# x = scaler.transform(x)
# scaler.fit(y.reshape(-1,1))
# y = scaler.transform(y.reshape(-1,1))



# findBestRandomState(100000,200000,x, y)
# seed 195043 MAE: 3452.587947630313
# seed 12049 MAE: 3531.341997248567
# seed 9930 MAE: 3576.2944849569312
# seed 607 MAE: 4145.711673614879

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=12049)

In [65]:
ml = RegressorChain(GradientBoostingRegressor(learning_rate=0.050000001, criterion='squared_error'))
ml.fit(x_train, y_train)

RegressorChain(base_estimator=GradientBoostingRegressor(criterion='squared_error',
                                                        learning_rate=0.050000001))

In [66]:
def mape(y_true, y_pred):
    return round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)

In [67]:
y_predict = ml.predict(x_test)
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_predict)}")
print(f"The Mean Square Error: {mean_squared_error(y_test, y_predict)}")
print(f"The R2 score: {r2_score(y_test, y_predict)}")


Mean Absolute Error: 3112.574600219415
The Mean Square Error: 14838757.278218528
The R2 score: 0.7033340519351896


In [68]:
print(ml.predict([[22243,22223,42342,46546,22425]]))

[[35393.50919725]]
